In [1]:
# %pip install sqlalchemy
# %pip install pymysql

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import ETL_Operations

In [3]:
# First we want to connect to the Data Warehouse server
engine_olap = create_engine('mysql+pymysql://root:''@localhost/sales_olap')
conn_olap = engine_olap.connect()

In [4]:
emp_df = ETL_Operations.emp_df.copy()

In [5]:
emp_df.head(10)

,emp_id,first_name,second_name,gender,age,hire_date,team_id,sup_first_name,sup_second_name,dept_name,manager_first_name,manager_second_name
0,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Lloyd,Booker
1,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Richard,Thomas
2,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Sean,Harvey
3,13,Dennis,Mcclanahan,M,45,2008-10-15,1,Armando,Green,Sales,Samuel,Chaffin
4,13,Dennis,Mcclanahan,M,45,2008-10-15,1,Armando,Green,Sales,Robert,Randell
5,18,Robert,Wilson,M,28,2001-08-21,2,Ryan,Curtis,Sales,Lloyd,Booker
6,18,Robert,Wilson,M,28,2001-08-21,2,Ryan,Curtis,Sales,Richard,Thomas
7,18,Robert,Wilson,M,28,2001-08-21,2,Ryan,Curtis,Sales,Sean,Harvey
8,19,Roger,Magaldi,M,37,2012-03-03,2,Ryan,Curtis,Sales,Eric,Moeller
9,19,Roger,Magaldi,M,37,2012-03-03,2,Ryan,Curtis,Sales,None,None


In [6]:
# From below we can see we have missing values
print(emp_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   emp_id               58 non-null     int64 
 1   first_name           58 non-null     object
 2   second_name          58 non-null     object
 3   gender               58 non-null     object
 4   age                  58 non-null     int64 
 5   hire_date            58 non-null     object
 6   team_id              58 non-null     object
 7   sup_first_name       41 non-null     object
 8   sup_second_name      41 non-null     object
 9   dept_name            58 non-null     object
 10  manager_first_name   52 non-null     object
 11  manager_second_name  52 non-null     object
dtypes: int64(2), object(10)
memory usage: 5.6+ KB
None


In [7]:
# We investigated and came to the conclusion that supervisors names are NULL if the employee is a manager and does not have one
# It is safe to replace the NULL values with 
emp_df.iloc[:]['sup_first_name'] = emp_df.iloc[:]['sup_first_name'].fillna("", inplace = True)
emp_df.iloc[:]['sup_second_name'] = emp_df.iloc[:]['sup_second_name'].fillna("", inplace = True)

# We can also change the NULL values in the managers fields because the NULL values indicate that one of the areas covered by the employees
# do not have a chief assigned
emp_df.iloc[:]['manager_first_name'] = emp_df.iloc[:]['manager_first_name'].fillna("No chief assigned", inplace = True)
emp_df.iloc[:]['manager_second_name'] = emp_df.iloc[:]['manager_second_name'].fillna("", inplace = True)

In [8]:
print(emp_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   emp_id               58 non-null     int64 
 1   first_name           58 non-null     object
 2   second_name          58 non-null     object
 3   gender               58 non-null     object
 4   age                  58 non-null     int64 
 5   hire_date            58 non-null     object
 6   team_id              58 non-null     object
 7   sup_first_name       58 non-null     object
 8   sup_second_name      58 non-null     object
 9   dept_name            58 non-null     object
 10  manager_first_name   58 non-null     object
 11  manager_second_name  58 non-null     object
dtypes: int64(2), object(10)
memory usage: 5.6+ KB
None


In [9]:
emp_df.head(5)

,emp_id,first_name,second_name,gender,age,hire_date,team_id,sup_first_name,sup_second_name,dept_name,manager_first_name,manager_second_name
0,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Lloyd,Booker
1,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Richard,Thomas
2,12,Gary,Smith,M,25,2014-10-18,1,Armando,Green,Sales,Sean,Harvey
3,13,Dennis,Mcclanahan,M,45,2008-10-15,1,Armando,Green,Sales,Samuel,Chaffin
4,13,Dennis,Mcclanahan,M,45,2008-10-15,1,Armando,Green,Sales,Robert,Randell


In [10]:
# Before going further, we want to merge the first and second names into one full name atttribute and also group rows by each emp_id

emp_df['employee_full_name'] = emp_df['first_name'] + ' ' + emp_df['second_name']
emp_df['supervisor_full_name'] = emp_df['sup_first_name'] + ' ' + emp_df['sup_second_name']
emp_df['manager_full_name'] = emp_df['manager_first_name'] + ' ' + emp_df['manager_second_name']
emp_df.drop(['first_name', 'second_name', 'sup_first_name', 'sup_second_name', 'manager_first_name', 'manager_second_name'], axis = 1, inplace = True)
emp_df.columns

Index(['emp_id', 'gender', 'age', 'hire_date', 'team_id', 'dept_name',
       'employee_full_name', 'supervisor_full_name', 'manager_full_name'],
      dtype='object')

In [11]:
emp_df.head()


,emp_id,gender,age,hire_date,team_id,dept_name,employee_full_name,supervisor_full_name,manager_full_name
0,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,Lloyd Booker
1,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,Richard Thomas
2,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,Sean Harvey
3,13,M,45,2008-10-15,1,Sales,Dennis Mcclanahan,Armando Green,Samuel Chaffin
4,13,M,45,2008-10-15,1,Sales,Dennis Mcclanahan,Armando Green,Robert Randell


In [12]:
# We now want to display all the managers in one single cell
emp_df['managers'] = emp_df.groupby(['emp_id'])['manager_full_name'].transform(lambda x : ', '.join(x))
emp_df.drop('manager_full_name', axis = 1, inplace = True)
emp_df.head()

,emp_id,gender,age,hire_date,team_id,dept_name,employee_full_name,supervisor_full_name,managers
0,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,"Lloyd Booker, Richard Thomas, Sean Harvey"
1,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,"Lloyd Booker, Richard Thomas, Sean Harvey"
2,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,"Lloyd Booker, Richard Thomas, Sean Harvey"
3,13,M,45,2008-10-15,1,Sales,Dennis Mcclanahan,Armando Green,"Samuel Chaffin, Robert Randell"
4,13,M,45,2008-10-15,1,Sales,Dennis Mcclanahan,Armando Green,"Samuel Chaffin, Robert Randell"


In [13]:
# Lets remove the duplicated rows so we will have only one record for each emp_id
emp_df.drop_duplicates(inplace = True)

In [14]:
# This shows that now we do not have duplicated rows anymore
emp_df.duplicated()

0     False
3     False
5     False
8     False
11    False
14    False
17    False
20    False
23    False
25    False
27    False
30    False
33    False
35    False
38    False
41    False
43    False
45    False
48    False
51    False
54    False
56    False
dtype: bool

In [15]:
# The emp_df looks already much cleaner. We now have to replace the "No team assigned No team assigned" values with simply "No team assigned"
emp_df

,emp_id,gender,age,hire_date,team_id,dept_name,employee_full_name,supervisor_full_name,managers
0,12,M,25,2014-10-18,1,Sales,Gary Smith,Armando Green,"Lloyd Booker, Richard Thomas, Sean Harvey"
3,13,M,45,2008-10-15,1,Sales,Dennis Mcclanahan,Armando Green,"Samuel Chaffin, Robert Randell"
5,18,M,28,2001-08-21,2,Sales,Robert Wilson,Ryan Curtis,"Lloyd Booker, Richard Thomas, Sean Harvey"
8,19,M,37,2012-03-03,2,Sales,Roger Magaldi,Ryan Curtis,"Eric Moeller, No chief assigned , Alberto Pence"
11,22,M,25,2008-06-23,2,Sales,Richard Pickett,Ryan Curtis,"Eric Moeller, No chief assigned , Alberto Pence"
14,14,M,45,2006-06-13,3,Sales,John Robinson,Barry Proctor,"Lloyd Booker, Richard Thomas, Sean Harvey"
17,15,M,26,2005-12-28,3,Sales,Joseph Spencer,Barry Proctor,"Eric Moeller, No chief assigned , Alberto Pence"
20,16,M,37,2016-02-14,3,Sales,Palmer Whitham,Barry Proctor,"Eric Moeller, No chief assigned , Alberto Pence"
23,21,M,41,2017-04-26,3,Sales,Arlen Wright,Barry Proctor,"Samuel Chaffin, Robert Randell"
25,17,M,41,2014-07-16,4,Sales,Denis Ramsey,Harold Spaulding,"Samuel Chaffin, Robert Randell"


In [16]:
# This replaces the empty cells from superviser_full_name with 'No team assigned'

emp_df['supervisor_full_name'].replace(to_replace = ' ', value = 'No team assigned', inplace = True)

# Lets rearrange the columns to correspond with the table employee created in the data warehouse system
emp_df = emp_df[['emp_id', 'employee_full_name', 'gender', 'age', 'hire_date', 'team_id', 'supervisor_full_name', 'dept_name', 'managers']]
emp_df

,emp_id,employee_full_name,gender,age,hire_date,team_id,supervisor_full_name,dept_name,managers
0,12,Gary Smith,M,25,2014-10-18,1,Armando Green,Sales,"Lloyd Booker, Richard Thomas, Sean Harvey"
3,13,Dennis Mcclanahan,M,45,2008-10-15,1,Armando Green,Sales,"Samuel Chaffin, Robert Randell"
5,18,Robert Wilson,M,28,2001-08-21,2,Ryan Curtis,Sales,"Lloyd Booker, Richard Thomas, Sean Harvey"
8,19,Roger Magaldi,M,37,2012-03-03,2,Ryan Curtis,Sales,"Eric Moeller, No chief assigned , Alberto Pence"
11,22,Richard Pickett,M,25,2008-06-23,2,Ryan Curtis,Sales,"Eric Moeller, No chief assigned , Alberto Pence"
14,14,John Robinson,M,45,2006-06-13,3,Barry Proctor,Sales,"Lloyd Booker, Richard Thomas, Sean Harvey"
17,15,Joseph Spencer,M,26,2005-12-28,3,Barry Proctor,Sales,"Eric Moeller, No chief assigned , Alberto Pence"
20,16,Palmer Whitham,M,37,2016-02-14,3,Barry Proctor,Sales,"Eric Moeller, No chief assigned , Alberto Pence"
23,21,Arlen Wright,M,41,2017-04-26,3,Barry Proctor,Sales,"Samuel Chaffin, Robert Randell"
25,17,Denis Ramsey,M,41,2014-07-16,4,Harold Spaulding,Sales,"Samuel Chaffin, Robert Randell"


In [17]:
# Now lets check the loccation dataframe
loc_df = ETL_Operations.loc_df.copy()
print(loc_df.info())
loc_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country_name   13 non-null     object
 1   city_name      13 non-null     object
 2   area_postcode  8 non-null      object
dtypes: object(3)
memory usage: 440.0+ bytes
None


,country_name,city_name,area_postcode
0,USA,New York,None
1,USA,Miami,HP510ES
2,USA,Miami,HP260ES
3,USA,San Francisco,None
4,Spain,Barcelona,None
5,Spain,Madrid,None
6,UK,London,None
7,UK,Manchester,LN831NE
8,UK,Manchester,LN208NE
9,UK,Manchester,LN382NE


In [18]:
loc_df.iloc[:]['area_postcode'] = loc_df.iloc[:]['area_postcode'].fillna('Branch in development', inplace = True)
print(loc_df.info())
loc_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country_name   13 non-null     object
 1   city_name      13 non-null     object
 2   area_postcode  13 non-null     object
dtypes: object(3)
memory usage: 440.0+ bytes
None


,country_name,city_name,area_postcode
0,USA,New York,Branch in development
1,USA,Miami,HP510ES
2,USA,Miami,HP260ES
3,USA,San Francisco,Branch in development
4,Spain,Barcelona,Branch in development
5,Spain,Madrid,Branch in development
6,UK,London,Branch in development
7,UK,Manchester,LN831NE
8,UK,Manchester,LN208NE
9,UK,Manchester,LN382NE


In [19]:
# Moving on to the property dataframe
prop_df = ETL_Operations.prop_df.copy()
print(prop_df.info())
prop_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prop_id      1000 non-null   int64 
 1   type_name    1000 non-null   object
 2   no_bed       1000 non-null   int64 
 3   no_bath      1000 non-null   int64 
 4   price        1000 non-null   int64 
 5   status_name  1000 non-null   object
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,prop_id,type_name,no_bed,no_bath,price,status_name
0,2,Bungalow,4,1,2150,LET AGREED
1,4,Bungalow,4,1,2150,LET AGREED
2,5,Bungalow,2,1,1150,LET AGREED
3,8,Land,0,0,433998,LET AGREED
4,10,Land,0,0,275623,LET AGREED


In [20]:
# Finally we look at the customer dataframe
custom_df = ETL_Operations.custom_df.copy()
print(custom_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   custom_id    200 non-null    int64 
 1   first_name   200 non-null    object
 2   second_name  200 non-null    object
 3   budget       200 non-null    int64 
 4   intention    200 non-null    object
dtypes: int64(2), object(3)
memory usage: 7.9+ KB
None


Now that our datasets are cleaned, readable and understandable, we can progress to the next stage which is loading them to the OLAP system

In [21]:
emp_df.to_sql('employee', conn_olap, if_exists = 'append', index = False)
loc_df.to_sql('location', conn_olap, if_exists = 'append', index = False)
prop_df.to_sql('property', conn_olap, if_exists = 'append', index = False)
custom_df.to_sql('customer', conn_olap, if_exists = 'append', index = False)

200

The final piece in the OLAP system is the fact table, which must contain historical data with regards to agreements. Therefore, from the  OLTP system have been extracted the properties which are "LET AGREED" or "SOLD", matching them with the best possible buyer or renter. The table extracted from the OLTP system is available below. The idea we going to work on is that a property is given to the customer who comes with highest budget. It was assumed that a customer may be involved with more than one agreement, but an agreement can belong to only one customer. Therefore, the agreement will be made based on the customer with the highest budget. However, it must be remembered that this is part of the ETL procedures and the client's budget will not be modified in the database because once an agreement is done, that respective customer is linked to its property and its details will be updated in the system once he wants to contact the company again for a further agreement.  

In [22]:
agreement_df = ETL_Operations.agreement_df.copy()

agreement_df

,custom_id,prop_id,loc_id,emp_id,tax_value,total_price,budget
0,40,5,1,17,57.5,1207.5,1227
1,142,5,1,17,57.5,1207.5,1176
2,170,5,1,17,57.5,1207.5,1346
3,175,5,1,17,57.5,1207.5,1445
4,3,37,3,14,32.5,682.5,1387
...,...,...,...,...,...,...,...
1507,47,1000,5,20,8250.0,173250.0,246896
1508,81,1000,5,20,8250.0,173250.0,788575
1509,92,1000,5,20,8250.0,173250.0,417420
1510,104,1000,5,20,8250.0,173250.0,837698


In [23]:
# Now we want to pick the customer with the highest budget for every property

agreement_df['max_budget'] = agreement_df.groupby(['prop_id'])['budget'].transform(lambda x: max(x))

In [24]:
agreement_df.head(30)

,custom_id,prop_id,loc_id,emp_id,tax_value,total_price,budget,max_budget
0,40,5,1,17,57.5,1207.5,1227,1445
1,142,5,1,17,57.5,1207.5,1176,1445
2,170,5,1,17,57.5,1207.5,1346,1445
3,175,5,1,17,57.5,1207.5,1445,1445
4,3,37,3,14,32.5,682.5,1387,1387
5,165,37,3,14,32.5,682.5,655,1387
6,168,37,3,14,32.5,682.5,968,1387
7,73,48,6,16,72.5,1522.5,1464,1464
8,66,51,5,20,57.5,1207.5,1424,1424
9,199,51,5,20,57.5,1207.5,1259,1424


In [25]:
# We want to filter the results and we will create a new dataframe to hold the agreements made
deal_df = pd.DataFrame(columns = agreement_df.columns)
print(deal_df)


for i in range(len(agreement_df.index)):
    if agreement_df.iloc[i][6] == agreement_df.iloc[i][7]:
        new_row = pd.DataFrame({'custom_id': int(agreement_df.iloc[i][0]), 'prop_id': int(agreement_df.iloc[i][1]),
                                'loc_id': int(agreement_df.iloc[i][2]), 'emp_id': int(agreement_df.iloc[i][3]),
                                'tax_value': agreement_df.iloc[i][4], 'total_price': agreement_df.iloc[i][5],
                                'budget': agreement_df.iloc[i][6], 'max_budget': agreement_df.iloc[i][7]}, index = [0])
        deal_df = pd.concat([deal_df, new_row], axis = 0, ignore_index = True)

deal_df

Empty DataFrame
Columns: [custom_id, prop_id, loc_id, emp_id, tax_value, total_price, budget, max_budget]
Index: []


,custom_id,prop_id,loc_id,emp_id,tax_value,total_price,budget,max_budget
0,175,5,1,17,57.5,1207.5,1445.0,1445.0
1,3,37,3,14,32.5,682.5,1387.0,1387.0
2,73,48,6,16,72.5,1522.5,1464.0,1464.0
3,66,51,5,20,57.5,1207.5,1424.0,1424.0
4,73,89,8,16,72.5,1522.5,1464.0,1464.0
...,...,...,...,...,...,...,...,...
303,91,964,2,21,16732.55,351383.55,711402.0,711402.0
304,2,965,1,17,35950.15,754953.15,899652.0,899652.0
305,136,990,6,16,28871.2,606295.2,888794.0,888794.0
306,136,999,6,16,22540.25,473345.25,888794.0,888794.0


In [26]:
#Now we will assume that each customer will go for the properties or property with the highest price

deal_df['cust_preference'] = deal_df.groupby(['custom_id'])['total_price'].transform(lambda x: max(x))


fact_df = pd.DataFrame(columns = agreement_df.columns)
#print(deal_df)


for i in range(len(deal_df.index)):
    if deal_df.iloc[i][5] == deal_df.iloc[i][8]:
        new_row = pd.DataFrame({'custom_id': int(deal_df.iloc[i][0]), 'prop_id': int(deal_df.iloc[i][1]),
                                'loc_id': int(deal_df.iloc[i][2]), 'emp_id': int(deal_df.iloc[i][3]),
                                'tax_value': deal_df.iloc[i][4], 'total_price': deal_df.iloc[i][5],
                                'budget': deal_df.iloc[i][6], 'max_budget': deal_df.iloc[i][7]}, index = [0])
        fact_df = pd.concat([fact_df, new_row], axis = 0, ignore_index = True)


print(fact_df)

   custom_id prop_id loc_id emp_id tax_value total_price    budget max_budget
0         73      48      6     16      72.5      1522.5    1464.0     1464.0
1         66      51      5     20      57.5      1207.5    1424.0     1424.0
2         73      89      8     16      72.5      1522.5    1464.0     1464.0
3        182      96      4     18      65.0      1365.0    1421.0     1421.0
4         54     143      7     22      72.5      1522.5    1481.0     1481.0
5        163     252      4     12      65.0      1365.0    1432.0     1432.0
6        182     416      5     18      65.0      1365.0    1421.0     1421.0
7         54     462      7     22      72.5      1522.5    1481.0     1481.0
8          3     577      5     14      65.0      1365.0    1387.0     1387.0
9         27     617      2     13      47.5       997.5    1043.0     1043.0
10       163     717      3     12      65.0      1365.0    1432.0     1432.0
11       175     843      1     17      65.0      1365.0    1445

In [27]:
# We do not need all the newly created columns so we will drop the last two columns to fit to the fact table previously created 
fact_df.drop(['budget', 'max_budget'], axis = 1, inplace = True)

fact_df

,custom_id,prop_id,loc_id,emp_id,tax_value,total_price
0,73,48,6,16,72.5,1522.5
1,66,51,5,20,57.5,1207.5
2,73,89,8,16,72.5,1522.5
3,182,96,4,18,65.0,1365.0
4,54,143,7,22,72.5,1522.5
5,163,252,4,12,65.0,1365.0
6,182,416,5,18,65.0,1365.0
7,54,462,7,22,72.5,1522.5
8,3,577,5,14,65.0,1365.0
9,27,617,2,13,47.5,997.5


In [28]:
# Now lets export the pre-processed dataset to the fact table in the data warehouse

fact_df.to_sql('agreement', con = conn_olap, if_exists = 'append', index = False)

28